In [18]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal, Annotated
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, BaseMessage
from langgraph.checkpoint.memory import MemorySaver

In [2]:
load_dotenv()

True

In [9]:
from langgraph.graph import add_messages
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [10]:
llm = ChatOpenAI()
def chat_node(state: ChatState):
    messages = state['messages']
    response = llm.invoke(messages)
    return {'messages': [response]}


In [19]:
# Create graph
checkpointer = MemorySaver()
graph = StateGraph(state_schema=ChatState)

# Add nodes
graph.add_node("chat_node", chat_node)

# add edges
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

# compile the graph
chatbot = graph.compile(checkpointer=checkpointer)

In [16]:
initial_state = {'messages': [HumanMessage("what is the capital of USA?")]}
final_state = chatbot.invoke(initial_state)
print(final_state)

{'messages': [HumanMessage(content='what is the capital of USA?', additional_kwargs={}, response_metadata={}, id='346ba444-7074-4326-8c0d-dcc2cdec9a9c'), AIMessage(content='The capital of the United States of America is Washington, D.C.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 14, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CA2a7avwdyWC9qb8tU3BdmUqXligz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--a068bd10-103f-4b06-b194-9a2bce74f7f1-0', usage_metadata={'input_tokens': 14, 'output_tokens': 14, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [20]:
thread_id = '1'
while True:
    user_message = input('type here: ')
    print('User: ', user_message)
    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break

    config = {'configurable': {'thread_id': thread_id}}
    response = chatbot.invoke({'messages':[HumanMessage(content=user_message)]}, config=config)
    print('AI: ', response['messages'][-1].content)


User:  Hi. my name is Prav
AI:  Hello Prav! Nice to meet you. How can I assist you today?
User:  what is my name?
AI:  Your name is Prav.
User:  what is 10 + 9
AI:  10 + 9 equals 19.
User:  now add 2 to the result
AI:  Adding 2 to the result of 10 + 9 (which is 19) equals 21.
User:  bye
